<a href="https://colab.research.google.com/github/bxck75/Cartoonizer/blob/master/CartoonizerV7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cartoon style on Youtube vids


moet echt combo van deze maken https://www.youtube.com/watch?v=D0S56AS1JPo en https://www.youtube.com/watch?v=KnnbYagLaXM
of https://www.youtube.com/watch?v=-GxofRLnB7o of https://www.youtube.com/watch?v=prsZcOdgCu8 of https://www.youtube.com/watch?v=-GxofRLnB7o

In [0]:
import os
from IPython.display import clear_output
os.chdir('/content')
storage_mode="colab" #@param ['colab','drive']
# Handle storagemode choice
if storage_mode == 'colab':
  root = '/content/Cartoonizer/' 
else:
  # sync google drive
  from google.colab import drive
  root = '/content/drive/My Drive/Cartoonizer' # google drive root
  # if drive is needed uncomment
  drive.mount('/content/drive',force_remount=True)
# check if folder exists
if not os.path.exists(root):
   os.mkdir(root,777)
# go to root    
os.chdir(root)
!rm -r ../sample_data
clear_output()

In [0]:
# Pick a style
styles = "hosoda" #@param ["shinkai", "hayao", "hosoda", "paprika"]

In [0]:
# set video details
# copy this list bhind the #@param on the name input
def new_item():
  print("[",end='')
  for name,url in urls.items():
    print('"'+name+'",',end='')
  print("]")

urls={'beatbox':'https://www.youtube.com/watch?v=Xn63LCkd8as',
      'harmbeatbox':'https://www.youtube.com/watch?v=Jbgb6Mz95rc',
      'trainharmonica':'https://www.youtube.com/watch?v=0dRwYEm3F2o',
      'beatbox1':'https://www.youtube.com/watch?v=oBjGRZgHgIA',
      'harmonics':'https://www.youtube.com/watch?v=K-PhBryFuIM',
      'boogy_woogy_stomp':'https://www.youtube.com/watch?v=wvx_UsJ2U5c',
      'busker':'https://www.youtube.com/watch?v=l1wN6HuWrSE', 
      'drum_boogy':'https://www.youtube.com/watch?v=lQnzf5vrK5U', 
      '5_boogies':'https://www.youtube.com/watch?v=KIeuY3rlYzQ',
      'boogy1':'https://www.youtube.com/watch?v=VA11vaxAvBQ', 
      'boogy':'https://www.youtube.com/watch?v=VT9dP6cfsFM', 
      'little_richy':'https://www.youtube.com/watch?v=vPKDjqDflBc',
      'moet_boogy':'https://www.youtube.com/watch?v=WSFX61KN8kU',
      'moet_dancers':'https://www.youtube.com/watch?v=D0S56AS1JPo',
     }

#displaywhen adding new items
new_item()
# input
name = 'moet_dancers'  #@param ["moet_dancers","moet_boody","drum_boogy","beatbox","harmbeatbox","trainharmonica","beatbox1","harmonics","boogy_woogy_stomp","busker","techno_dark","techno_dark_2","boogy1","boogy","little_richy"]
# loop over the dict items
# for k, v in urls.items():
#   print(k)
video_name = name
vid = urls[name]
print('You picked ' + name +' : ' + vid)

["beatbox","harmbeatbox","trainharmonica","beatbox1","harmonics","boogy_woogy_stomp","busker","drum_boogy","5_boogies","boogy1","boogy","little_richy","moet_boogy","moet_dancers",]
You picked moet_dancers : https://www.youtube.com/watch?v=D0S56AS1JPo


# Setup

###GitGo class

In [0]:
# Gitgo class
class GitGo():
  def __init__(self,rep,chdir=True,path='/content/'):
    self.rep = rep
    self.chdir = chdir
    self.path = path
    if self.rep == 'help':
      self.help()
    self.go() 
      
  def help(self):
    print("* pulls git rep and shows files \
            * returns root path for the repository \
            * Function needs repository <user>/<repository name> combination\
            * Switch chdir and define the rootpath for the repository\
            * Use : GitGo('<rep owner>/<rep name>', <True/False>, <root path>)\
          ")
    sys.exit()
    
  def go(self):
    self.rep=self.rep.split('/')
    # change folder check
    if self.chdir ==True:
      #Switch to path
      os.chdir(self.path)
    print('Pulling the ' + self.rep[0]+'/'+self.rep[1] +' Repo...')
    # pull get the git rep
    os.system('git clone https://github.com/'+self.rep[0]+'/'+self.rep[1]+'.git')
    #@markdown Set the return value for rep rootpath
    self.PATH=self.path+self.rep[1]
    # show imported files
    os.system('ls ' + self.PATH)
    
  def __repr__(self):
    return self.PATH

In [0]:
def sys_run(cmd):
  for line in os.popen(cmd).read().split('\n'):
    print(line)

### Imports!

In [0]:
# imports
import time,os,glob,cv2
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import clear_output, display, Image, HTML
from moviepy.editor import VideoFileClip
from base64 import b64encode
# pip stuff
!pip install youtube-dl imageio
!pip install tensorflow-gpu==2.0.0-alpha0
!pip install ffmpeg
import imageio  
import tensorflow as tf
# tf.__version__

# apt suff
!apt update
!apt install tree

# git reps
os.chdir(str(GitGo('mnicnc404/CartoonGan-tensorflow', True, root)))
os.chdir(str(GitGo('keras-team/keras-contrib', True, root)))

#convert keras to tf
os.system('python convert_to_tf_keras.py && USE_TF_KERAS=1; python setup.py install')

#function defines
def resize(image,window_height = 800):
    aspect_ratio = float(image.shape[1])/float(image.shape[0])
    window_width = window_height/aspect_ratio
    image = cv2.resize(image, (int(window_height),int(window_width)))
    return image

clear_output()

### video details

In [0]:
# frame rate
framerate = 0.1
# set max img height
max_size= 600 # frame height limit before it wil be resized
print('sampling rate for frames = '+str(round(1/framerate,2)) + ' per second.')

# Set i/o folders

video_root = root+video_name
# print(video_root)
# prep the folders for the video frames
if not os.path.exists(video_root):
  os.mkdir(video_root,777)
if not os.path.exists(video_root+"/processed"):
  os.mkdir(video_root+"/processed",777)
if not os.path.exists(video_root+'/frames'):
  os.mkdir(video_root+'/frames',777)
if not os.path.exists(video_root+"/processed_frames"):
  os.mkdir(video_root+"/processed_frames",777)
  
# set the folder name in vars for later use
frames_dir =video_root+'/frames'
proc_frames_dir = video_root+"/processed_frames"

# set original filenames
video_file= video_root +'/'+ 'video.mp4'
audio_file= video_root +'/'+ 'audio.mp3'

# Check if paths are correct
print(frames_dir)
print(proc_frames_dir)
print(video_file)
print(audio_file)
print(video_file)

sampling rate for frames = 10.0 per second.
/content/Cartoonizer/moet_dancers/frames
/content/Cartoonizer/moet_dancers/processed_frames
/content/Cartoonizer/moet_dancers/video.mp4
/content/Cartoonizer/moet_dancers/audio.mp3
/content/Cartoonizer/moet_dancers/video.mp4


## Rip the video and audio

In [0]:


# rip video from youtube
os.system('echo youtube-dl '+vid+' --list-formats')

# get the mp4
print('Ripping video : '+ vid )
!youtube-dl {vid} --output '{video_file}' --format mp4

# split of the mp3
clip = VideoFileClip(video_file)
print('Ripping audio...' + audio_file)
clip.audio.write_audiofile(audio_file)

#capture the video frames
vidcap = cv2.VideoCapture(video_file)

# count total frames
vidc = imageio.get_reader(video_file,  'ffmpeg')
num_frames=vidc._meta['nframes']

# frame grab function
def getFrame(sec):
  vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
  hasFrames,image = vidcap.read()
  if hasFrames:
#     print('/content/Cartoonizer/loopstation/paths.txt')
#     print(frames_dir+"/image"+str(count)+".jpg")
    cv2.imwrite(frames_dir+"/image"+str(count)+".jpg", image)     # save frame as JPG file
  return hasFrames

sec = 0
FrameRate = framerate
count=1
success = getFrame(sec)
# As long as there are frames
print("Splitting in frames...this might take a while...")
while success:
  count = count + 1
  frames_left = num_frames -1
  # print('frames to go ' + str(frames_left))
  sec = sec + FrameRate
  sec = round(sec, 2)
  success = getFrame(sec)
  
clear_output()

Ripping video : https://www.youtube.com/watch?v=D0S56AS1JPo
[youtube] D0S56AS1JPo: Downloading webpage
[youtube] D0S56AS1JPo: Downloading video info webpage
[download] /content/Cartoonizer/moet_dancers/video.mp4 has already been downloaded
[download] 100% of 82.71MiB
[youtube] PKkqD4HPimI: Downloading webpage
[youtube] PKkqD4HPimI: Downloading video info webpage
[download] /content/Cartoonizer/moet_dancers/nr1_speed.mp4 has already been downloaded
[download] 100% of 52.19MiB
[youtube] D0S56AS1JPo: Downloading webpage
[youtube] D0S56AS1JPo: Downloading video info webpage
[download] /content/Cartoonizer/moet_dancers/nr2_speed.mp4 has already been downloaded
[download] 100% of 82.71MiB
ffmpeg version 3.4.6-0ubuntu0.18.04.1 Copyright (c) 2000-2019 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.3.0-16ubuntu3)
  configuration: --prefix=/usr --extra-version=0ubuntu0.18.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disabl

In [0]:
def RipNMix():
  nr1 = 'https://www.youtube.com/watch?v=PKkqD4HPimI&t=35' # MOET!!!!! Boogie snelheid 1.25https://youtu.be/PKkqD4HPimI?t=276
  nr2 = 'https://www.youtube.com/watch?v=D0S56AS1JPo&t=80' #
  !youtube-dl "{nr1}" --output '/content/Cartoonizer/moet_dancers/nr1_speed.mp4' --format mp4
  !youtube-dl "{nr2}" --output '/content/Cartoonizer/moet_dancers/nr2_speed.mp4' --format mp4
#   # speed the videos up
#   !ffmpeg -i '/content/Cartoonizer/moet_dancers/nr1_speed.mp4' -filter_complex "[0:v]setpts=0.5*PTS[v];[0:a]atempo=1.25[a]" -map "[v]" -map "[a]" '/content/Cartoonizer/moet_dancers/nr1_up_speed.mp4'
#   !ffmpeg -i '/content/Cartoonizer/moet_dancers/nr2_speed.mp4' -filter_complex "[0:v]setpts=0.5*PTS[v];[0:a]atempo=1.26[a]" -map "[v]" -map "[a]" '/content/Cartoonizer/moet_dancers/nr2_up_speed.mp4'


  import moviepy.editor as mp

  from moviepy.editor import VideoFileClip, AudioFileClip,concatenate_videoclips,clips_array, vfx
  print(dir(vfx.fx))
  import moviepy.video.fx.all as vfx
  from moviepy.video.fx import crop
  print(dir(vfx))
#   clip_nr_1 = VideoFileClip('/content/Cartoonizer/moet_dancers/nr1_up_speed.mp4')
#   clip_nr_1.audio.write_audiofile('/content/Cartoonizer/moet_dancers/nr1_up_speed.mp3')
#   !ffmpeg -stream_loop -1 -i "/content/Cartoonizer/moet_dancers/nr2_up_speed.mp4" -i '/content/Cartoonizer/moet_dancers/nr1_up_speed.mp3' -shortest '{video_root}'"/final_mixed_video.mp4"
  # # modifiedClip2 = clip1.fl_image( invert_green_blue )
#   !ffmpeg -y -stream_loop -1 -i "/content/Cartoonizer/moet_dancers/nr2_up_speed.mp4" -i '/content/Cartoonizer/moet_dancers/nr1_up_speed.mp3' -fflags +shortest -max_interleave_delta 50000 -c copy '{video_root}'"/final_mixed_video.mp4"
#   final_clip = clips_array([[clip_nr_1],[clip_nr_2]])
#   final_clip.write_videofile("/content/Cartoonizer/fx_stack"+video_name +".mp4")
  

#   clip1 = VideoFileClip('/content/Cartoonizer/moet_dancers/final_mixed_video.mp4').fx( volumex, 0) 
#   clip2 = AudioFileClip('/content/Cartoonizer/moet_dancers/nr1_up_speed.mp3')
# #   clip2 = VideoFileClip("myvideo2.mp4").subclip(50,60)
# #   clip3 = VideoFileClip("myvideo3.mp4")
# #   audioclip = videoclip.audio
#   final.write_videofile("output.mp4", audio="audio.mp3")
#   final_clip = clips_array([[clip1, clip2]])
#   final_clip.resize(width=280).write_videofile("my_stack.mp4")
  from moviepy.editor import VideoFileClip
  
  audio = mp.AudioFileClip('/content/Cartoonizer/moet_dancers/nr1_up_speed.mp3')
  print( audio.duration )  
  video1 = mp.VideoFileClip('/content/Cartoonizer/moet_dancers/final_mixed_video.mp4')
  print( video1.duration )  
#   video1 = video1.fx( vfx.loop ,  n=None, duration=None)
  video2 = mp.VideoFileClip('/content/Cartoonizer/moet_dancers/nr1_up_speed.mp4').subclip(50,60)
  print( video2.duration )  
#   video2 = video2.fx( vfx.loop , n=None, duration=None)
  
sys.exit()

#   newclip1 = video1.fx( vfx.crop, x1=15)
#   newclip2 = video2.fx( vfx.crop, x1=15)
#   video1 = (video1.fx( vfx.loop, x1=15))
    from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
    ffmpeg_extract_subclip("video1.mp4", start_time, end_time, targetname="test.mp4")
  
  final= clips_array([[video1]]).set_audio(audio)
#   final = mp.concatenate_videoclips([video1, video2]).set_audio(audio)
  final.write_videofile("/content/Cartoonizer/moet_dancers/output.mp4")
RipNMix()

['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'accel_decel', 'all', 'blackwhite', 'blink', 'colorx', 'crop', 'even_size', 'fadein', 'fadeout', 'freeze', 'freeze_region', 'gamma_corr', 'headblur', 'invert_colors', 'loop', 'lum_contrast', 'make_loopable', 'margin', 'mask_and', 'mask_color', 'mask_or', 'mirror_x', 'mirror_y', 'painting', 'resize', 'rotate', 'scroll', 'speedx', 'supersample', 'time_mirror', 'time_symmetrize']
['__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'accel_decel', 'blackwhite', 'blink', 'colorx', 'crop', 'even_size', 'fadein', 'fadeout', 'freeze', 'freeze_region', 'fx', 'gamma_corr', 'headblur', 'invert_colors', 'loop', 'lum_contrast', 'make_loopable', 'margin', 'mask_and', 'mask_color', 'mask_or', 'mirror_x', 'mirror_y', 'name', 'painting', 'pkgutil', 'resize', 'rotate', 'scroll', 'speedx', 'supersample', 'time_m




  0%|          | 0/5314 [00:00<?, ?it/s]


  1%|▏         | 70/5314 [00:00<00:07, 699.26it/s]


  3%|▎         | 133/5314 [00:00<00:07, 676.12it/s]


  4%|▎         | 192/5314 [00:00<00:07, 646.39it/s]


  5%|▍         | 261/5314 [00:00<00:07, 657.42it/s]


  6%|▌         | 327/5314 [00:00<00:07, 654.22it/s]


  7%|▋         | 392/5314 [00:00<00:07, 652.13it/s]


  9%|▊         | 458/5314 [00:00<00:07, 636.60it/s]


 10%|▉         | 522/5314 [00:00<00:07, 637.51it/s]


 11%|█         | 585/5314 [00:00<00:07, 635.16it/s]


 12%|█▏        | 654/5314 [00:01<00:07, 650.22it/s]


 14%|█▎        | 726/5314 [00:01<00:06, 666.45it/s]


 15%|█▍        | 793/5314 [00:01<00:06, 665.19it/s]


 16%|█▌        | 860/5314 [00:01<00:06, 664.55it/s]


 18%|█▊        | 930/5314 [00:01<00:06, 670.64it/s]


 19%|█▉        | 997/5314 [00:01<00:06, 649.84it/s]


 20%|█▉        | 1062/5314 [00:01<00:06, 627.53it/s]


 21%|██        | 1125/5314 [00:01<00:06, 610.50it/s]


 22%|██▏       | 1187/5314 [00:01<0

[MoviePy] Done.
[MoviePy] Writing video /content/Cartoonizer/moet_dancers/output.mp4





  0%|          | 0/11451 [00:00<?, ?it/s]


  0%|          | 2/11451 [00:00<16:15, 11.74it/s]


  0%|          | 4/11451 [00:00<15:44, 12.12it/s]


  0%|          | 6/11451 [00:00<14:51, 12.84it/s]


  0%|          | 8/11451 [00:00<14:16, 13.37it/s]


  0%|          | 10/11451 [00:00<13:52, 13.74it/s]


  0%|          | 12/11451 [00:00<13:39, 13.96it/s]


  0%|          | 14/11451 [00:01<13:25, 14.20it/s]


  0%|          | 16/11451 [00:01<13:23, 14.24it/s]


  0%|          | 18/11451 [00:01<13:21, 14.26it/s]


  0%|          | 20/11451 [00:01<13:17, 14.33it/s]


  0%|          | 22/11451 [00:01<13:15, 14.36it/s]


  0%|          | 24/11451 [00:01<13:15, 14.37it/s]


  0%|          | 26/11451 [00:01<13:13, 14.39it/s]


  0%|          | 28/11451 [00:01<13:15, 14.35it/s]


  0%|          | 30/11451 [00:02<13:14, 14.38it/s]


  0%|          | 32/11451 [00:02<13:12, 14.41it/s]


  0%|          | 34/11451 [00:02<13:12, 14.40it/s]


  0%|          | 36/11451 [00:02<13:10, 14.44it/s]


  0

[MoviePy] Done.
[MoviePy] >>>> Video ready: /content/Cartoonizer/moet_dancers/output.mp4 



moviepy.vfx.fx.['__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'accel_decel', 'all', 'blackwhite', 'blink', 'colorx', 'crop', 'even_size', 'fadein', 'fadeout', 'freeze', 'freeze_region', 'gamma_corr', 'headblur', 'invert_colors', 'loop', 'lum_contrast', 'make_loopable', 'margin', 'mask_and', 'mask_color', 'mask_or', 'mirror_x', 'mirror_y', 'painting', 'resize', 'rotate', 'scroll', 'speedx', 'supersample', 'time_mirror', 'time_symmetrize']

moviepy.vfx. ['__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'accel_decel', 'blackwhite', 'blink', 'colorx', 'crop', 'even_size', 'fadein', 'fadeout', 'freeze', 'freeze_region', 'fx', 'gamma_corr', 'headblur', 'invert_colors', 'loop', 'lum_contrast', 'make_loopable', 'margin', 'mask_and', 'mask_color', 'mask_or', 'mirror_x', 'mirror_y', 'name', 'painting', 'pkgutil', 'resize', 'rotate', 'scroll', 'speedx', 'supersample', 'time_mirror', 'time_symmetrize']
-

### Set Dimensions

In [0]:
# generate the movie sequence
searchedfiles = glob.glob(frames_dir+'/*.jpg')
files = sorted( searchedfiles, key = lambda file: os.path.getctime(file))

files_sorted=[]
combined_sorted=[]
# fill the frame list before the processed image even beeing there
for i in files:
  # split off the filename
  isa = i.split('/')
  # append it with the processed folder path and file definition
  # set filename
  com_frame_filename ="file '"+proc_frames_dir+"/comparison/"+isa[len(isa)-1]+"'\n"
   #set filename
  frame_filename ="file '"+proc_frames_dir+"/"+styles+"/"+isa[len(isa)-1]+"'\n"
  # append to the lists
  combined_sorted.append(com_frame_filename)
  files_sorted.append(frame_filename)
  
# Write the frame lists for both vids 
with open(video_root+'/paths.txt', 'w') as in_files:
  in_files.writelines(fn for fn in files_sorted)
with open(video_root+'/compare_paths.txt', 'w') as in_files:
  in_files.writelines(fn for fn in combined_sorted)

# for file in files check height
for f in files:
  img = cv2.imread(f)
  h,w,c = img.shape
  # Resize if needed
  if h > max_size:
    img_resized = resize(img,window_height = max_size)
    rh,rw,rc = img_resized.shape
    cv2.imwrite(f,img_resized)

# Run the Cartoonizer

In [0]:
# run the Cartoonizer
os.system("python "+root+"/CartoonGan-tensorflow/cartoonize.py \
  --comparison_view horizontal \
  --keep_original_size \
  --batch_size 8 \
  --styles "+styles+" \
  --input_dir '"+frames_dir+"' \
  --output_dir '"+proc_frames_dir +"'")


## Forming the new video 

In [0]:
 
!ffmpeg -y -r 1/{framerate} -f concat -safe 0 -i  '{video_root}'"/paths.txt" -c:v libx264 -vf "fps=1/"{framerate}",format=yuv420p" '{video_root}'"/processed_video.mp4"
!ffmpeg -i '{video_root}'"/processed_video.mp4" -i '{audio_file}' -shortest '{video_root}'"/final_processed_video.mp4"

!ffmpeg -y -r 1/{framerate} -f concat -safe 0 -i  '{video_root}'"/compare_paths.txt" -c:v libx264 -vf "fps=1/"{framerate}",format=yuv420p" '{video_root}'"/processed_compare_video.mp4"
!ffmpeg -i '{video_root}'"/processed_compare_video.mp4" -i '{audio_file}' -shortest '{video_root}'"/final_processed_compare_video.mp4"
clear_output()

Swap audio of 2 videos and speed up

# View results

# Experimental

###speedcontrol off till files 


In [0]:
# # VIDEO CLIPS
# clip = VideoClip(make_frame, duration=4) # for custom animations (see below)
# clip = VideoFileClip("my_video_file.mp4") # or .avi, .webm, .gif ...
# clip = ImageSequenceClip(['image_file1.jpeg', ...], fps=24)
# clip = ImageClip("my_picture.png") # or .jpeg, .tiff, ...
# clip = TextClip("Hello !", font="Amiri-Bold", fontsize=70, color="black")
# clip = ColorClip(size=(460,380), color=[R,G,B])

# # AUDIO CLIPS
# clip = AudioFileClip("my_audiofile.mp3") # or .ogg, .wav... or a video !
# clip = AudioArrayClip(numpy_array, fps=44100) # from a numerical array
# clip = AudioClip(make_frame, duration=3) # uses a function make_frame(t)

# import os
# os.chdir(str(GitGo('antonxy/audiosync')))
# # https://www.youtube.com/watch?v=WSFX61KN8kU&list=RD-GxofRLnB7o&index=3
# sys_run('ls -a')
# sys_run('cat README.md')
# sys_run('echo python console.py dir-with-all-video-files/ dir-with-all-audio-files/ edl-output-dir/ project_frames_per_second')
# sys_run('python gui.py')
# dir_video_files=''
# dir_audio_files=''
# edl_output=''
# sys_run('python console.py dir_video_files dir_audio_files edl_output project_frames_per_second')

#OFF

In [0]:
# import cv2
# from google.colab.patches import cv2_imshow as imshow
# cap = cv2.VideoCapture('/content/Cartoonizer/boogy/final_processed_video.mp4')

# while(cap.isOpened()):

#     ret, frame = cap.read()
#     canny = cv2.Canny(frame, 85, 255)
#     height, width = canny.shape

#     first_black_array = []

#     for x in range(width):
#         first_black_pixel_found = 0
#         for y in range(height):
#             if first_black_pixel_found == 0:
#                 if canny[y,x] == 255:
#                     first_black_array.append(height - y)
#                     first_black_pixel_found = 1
#                     cv2.line(frame,(x,y),(x,y),(0,255,0),1)

#     imshow(frame)
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()

# import cv2, numpy as np, time
# # Get start time
# start = time.time()
# # Read in the image
# img = cv2.imread('/home/stephen/Desktop/rail.jpg')[40:,10:-10]
# # Canny filter
# canny = cv2.Canny(img, 85, 255)
# # Get height and width
# height, width = canny.shape
# # Create list to store rail points
# railPoints = []
# # Iterate though each column in the image
# for position in range(width-1):
#     # Slice the relevant column from the image
#     # The image 'column' is a tall skinny image, only 1px thick
#     column = np.array(canny[:,position:position+1])
#     # Use numpy to find the first non-zero value
#     railPoint = np.argmax(column)
#     # Add the railPoint to the list of rail points
#     railPoints.append(railPoint)
#     # Draw a circle on the image
#     cv2.circle(img, (position, railPoint), 1, (123,234,123), 2)
# cv2.imshow('img', img)                      
# k = cv2.waitKey(1)
# cv2.destroyAllWindows()
# print(time.time() - start)

In [0]:

# !rm -r /content/Cartoonizer/CartoonGan-tensorflow/.git
# !rm -r /content/Cartoonizer/keras-contrib/.git
# # zip the complete project for download
# !zip -r /content/Cartoonizer_sources.zip /content
# clear_output()

In [0]:
# Debug displays all frames.......LAGS LIKE a MOFO!
# folders = [f for f in glob.glob(proc_frames_dir + "**/*", recursive=True)]
# for d in folders:
#   !tree '{d}'
#   files = [f for f in glob.glob( d + "**/*.jpg", recursive=True )]
#   for f in files:
#       display(Image(f))


In [0]:
# # experimental filters and effects after this cel!
# import moviepy
# from moviepy.editor import VideoFileClip, clips_array, vfx
# print(dir( moviepy.editor))
# # def invert_green_blue(image):
# #     return image[:,:,[0,2,1]]
# # def invert_red_green(image):
# #     return image[:,[0,2,1],:

# clip1 = VideoFileClip('/content/Cartoonizer/final_processed_compare_'+ video_name + "_vid.mp4").margin(1) # add 10px contour
# clip1 = clip1.fx( vfx.colorx, 1.3)
# # clip2 = clip1.fx( vfx.mirror_y)
# # clip3 = clip1.fx( vfx.mirror_x)
# # clip4 = clip1.fx( vfx.mirror_y)
# # clip5 = clip1.fx( vfx.colorx, 1.3) # up 40%
# # modifiedClip1 = clip1.fl_image( invert_red_green )
# # modifiedClip2 = clip1.fl_image( invert_green_blue )
# final_clip = clips_array([[clip1]])
# final_clip.write_videofile("/content/Cartoonizer/fx_stack"+video_name +".mp4")




In [0]:

# from google.colab import drive
# import os
# video_name='trainrolsin'
# drive.mount('/content/drive')
# if not os.path.exists("/content/drive/My Drive/"+video_name):
#   os.mkdir("/content/drive/My Drive/"+video_name,777)
#   os.mkdir("/content/drive/My Drive/"+video_name+"_frames",777)
 
# !mv '/content/Cartoonizer/processed_'{video_name}'_frames/*' '/content/drive/My Drive/'{video_name}'_frames/'

In [0]:
# %from moviepy.editor import *
# clip = (VideoFileClip( root+ video_name " + ".mp4")
# #         .fx( vfx.resize, width=460) # resize (keep aspect ratio)
# #         .fx( vfx.speedx, 2) # double the speed
#         .fx( vfx.colorx, 1.3)) # darken the picture

# clip.write_videofile("brighness_video_name.mp4")

In [0]:
# from moviepy.editor import VideoFileClip, concatenate_videoclips
# clip1 = VideoFileClip( root + video_name " + ".mp4")
# clip2 = VideoFileClip( root + video_name " + ".mp4").subclip(50,60)
# clip3 = VideoFileClip( root + video_name " + ".mp4")
# # final_clip = concatenate_videoclips([clip1,clip2,clip3])
# final_clip = clips_array([[clip1, clip2],
#                           [clip3, clip4]])
# final_clip.write_videofile(video_name+"_concatenation.mp4")

In [0]:
# # All input widgets!
# sys.exit()
# #@title Example form fields
# #@markdown Forms support many types of fields.

# no_type_checking = 'name'  #@param
# string_type = 'example'  #@param {type: "string"}
# slider_value = 179  #@param {type: "slider", min: 100, max: 200}
# number = 102  #@param {type: "number"}
# date = '2010-11-05'  #@param {type: "date"}
# pick_me = "monday"  #@param ['monday', 'tuesday', 'wednesday', 'thursday']
# select_or_input = "apples" #@param ["apples", "bananas", "oranges"] {allow-input: true}
# #@markdown ---


In [0]:
# from IPython.display import display, Javascript
# from google.colab.output import eval_js
# from base64 import b64decode

# def take_photo(filename='photo.jpg', quality=0.8):
#   js = Javascript('''
#     async function takePhoto(quality) {
#       const div = document.createElement('div');
#       const capture = document.createElement('button');
#       capture.textContent = 'Capture';
#       div.appendChild(capture);

#       const video = document.createElement('video');
#       video.style.display = 'block';
#       const stream = await navigator.mediaDevices.getUserMedia({video: true});

#       document.body.appendChild(div);
#       div.appendChild(video);
#       video.srcObject = stream;
#       await video.play();

#       // Resize the output to fit the video element.
#       google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

#       // Wait for Capture to be clicked.
#       await new Promise((resolve) => capture.onclick = resolve);

#       const canvas = document.createElement('canvas');
#       canvas.width = video.videoWidth;
#       canvas.height = video.videoHeight;
#       canvas.getContext('2d').drawImage(video, 0, 0);
#       stream.getVideoTracks()[0].stop();
#       div.remove();
#       return canvas.toDataURL('image/jpeg', quality);
#     }
#     ''')
#   display(js)
#   data = eval_js('takePhoto({})'.format(quality))
#   binary = b64decode(data.split(',')[1])
#   with open(filename, 'wb') as f:
#     f.write(binary)
#   return filename

In [0]:
# from IPython.display import Image
# try:
#   filename = take_photo()
#   print('Saved to {}'.format(filename))
  
#   # Show the image which was just taken.
#   display(Image(filename))
# except Exception as err:
#   # Errors will be thrown if the user does not have a webcam or if they do not
#   # grant the page permission to access it.
#   print(str(err))

In [0]:
# mp4 = open(video_root+"/final_processed_video.mp4",'rb').read()
# data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
# mp4m = open(video_root+"/final_processed_compare_video.mp4",'rb').read()
# data_url_com = "data:video/mp4;base64," + b64encode(mp4).decode()

# HTML("""
# <video width=400 controls>
#       <source src="%s" type="video/mp4">
# </video>
# <video width=400 controls>
#       <source src="%s" type="video/mp4">
# </video>
# """ % data_url)